In [1]:
!pip install transformers[torch] tokenizers datasets evaluate rouge_score sentencepiece huggingface_hub --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.9/7.9 MB 51.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 105.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 59.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 91.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.2/311.2 kB 37.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 85.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 261.4/261.4 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 18.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 20.5 MB/s eta 0:00:00
  Created wheel for rouge_score: filename=rouge_score-0.1.2-py3-none-any.whl size=24933 sha256=b54d470fe3f59ba4301668133af4c8e6d8a48b7c9a2a7eb1916

### Import Libraries

In [2]:
!pip install  tensorboard py7zr --upgrade

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 11.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 kB 8.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 22.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 412.3/412.3 kB 20.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.9/138.9 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.7/49.7 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 93.1/93.1 kB 14.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.6 MB/s eta 0:00:00
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.14.1
    Uninstalling tensorboard-2.14.1:
      Successfully uninstalled tensorboard-2.14.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorf

In [3]:
!sudo apt-get install git-lfs --yes

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 8 not upgraded.


In [4]:
# Import the necessary libraries
import nltk
from datasets import load_dataset
import evaluate
import numpy as np
from transformers import T5Tokenizer, DataCollatorForSeq2Seq
from transformers import T5ForConditionalGeneration, Seq2SeqTrainingArguments, Seq2SeqTrainer

### Load and Split Dataset

In [5]:
# Load and split the dataset
dataset = load_dataset("Omkar7/Medical_data")
dataset = dataset["train"].train_test_split(test_size=0.2)

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
dataset['train'][0]

{'question': 'How many people are affected by Prostate Cancer ?',
 'answer': 'Prostate cancer is one of the most common types of cancer among American men. It is a slow-growing disease that mostly affects older men. In fact, more than 60 percent of all prostate cancers are found in men over the age of 65. The disease rarely occurs in men younger than 40 years of age.'}

In [8]:
print(f"Train dataset size: {len(dataset['train'])}")
print(f"Test dataset size: {len(dataset['test'])}")

Train dataset size: 421
Test dataset size: 106


### Tokenization and Model Initialization

In [7]:
# Load the tokenizer, model, and data collator
tokenizer = T5Tokenizer.from_pretrained("google/flan-t5-base")
model = T5ForConditionalGeneration.from_pretrained("google/flan-t5-base")
data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

(…)-base/resolve/main/tokenizer_config.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…)ase/resolve/main/special_tokens_map.json:   0%|          | 0.00/2.20k [00:00<?, ?B/s]

(…)flan-t5-base/resolve/main/tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thouroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


(…)le/flan-t5-base/resolve/main/config.json:   0%|          | 0.00/1.40k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

(…)base/resolve/main/generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

### Preprocessing

In [9]:
# We prefix our tasks with "answer the question"
prefix = "answer the question: "

# Define our preprocessing function
def preprocess_function(examples):
    """Add prefix to the sentences, tokenize the text, and set the labels"""
    # The "inputs" are the tokenized answer:
    inputs = [prefix + doc for doc in examples["question"]]
    model_inputs = tokenizer(inputs, max_length=128, truncation=True)

    # The "labels" are the tokenized outputs:
    labels = tokenizer(text_target=examples["answer"], max_length=512, truncation=True)
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [10]:
# Map the preprocessing function across our dataset
tokenized_dataset = dataset.map(preprocess_function, batched=True)

Map:   0%|          | 0/421 [00:00<?, ? examples/s]

Map:   0%|          | 0/106 [00:00<?, ? examples/s]

### Evaluation Setup & Metrics Computation


In [11]:
#Set up Rouge score for evaluation
nltk.download("punkt", quiet=True)
metric = evaluate.load("rouge")

def compute_metrics(eval_preds):
    preds, labels = eval_preds

    # decode preds and labels
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # rougeLSum expects newline after each sentence
    decoded_preds = ["\n".join(nltk.sent_tokenize(pred.strip())) for pred in decoded_preds]
    decoded_labels = ["\n".join(nltk.sent_tokenize(label.strip())) for label in decoded_labels]

    result = metric.compute(predictions=decoded_preds, references=decoded_labels, use_stemmer=True)
    return result

### Training Setup & Trainer Initialization and Training

In [12]:
# Set up training arguments
training_args = Seq2SeqTrainingArguments(
    output_dir="./finetuned-t5-medical",
    evaluation_strategy="epoch",
    learning_rate=3e-4,
    per_device_train_batch_size=5,
    per_device_eval_batch_size=5,
    logging_dir="./finetuned-t5-medical/logs",
    logging_strategy="steps",
    logging_steps=500,
    weight_decay=0.01,
    save_total_limit=5,
    num_train_epochs=5,
    predict_with_generate=True,
    push_to_hub=False,
    report_to="tensorboard"
)

# Set up trainer
trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset["train"],
    eval_dataset=tokenized_dataset["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics
)

In [13]:
# Train the model
trainer.train()

Epoch,Training Loss,Validation Loss,Rouge1,Rouge2,Rougel,Rougelsum
1,No log,2.049878,0.118627,0.030668,0.094008,0.107069
2,No log,1.946286,0.102109,0.025099,0.083846,0.092616
3,No log,1.894242,0.115847,0.032376,0.093368,0.108289
4,No log,1.870630,0.110079,0.027867,0.085955,0.101679
5,No log,1.868590,0.111365,0.030354,0.087192,0.101992


/usr/local/lib/python3.10/dist-packages/transformers/generation/utils.py:1273: UserWarning: Using the model-agnostic default `max_length` (=20) to control the generation length. We recommend setting `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(


TrainOutput(global_step=425, training_loss=1.9712982536764705, metrics={'train_runtime': 454.7666, 'train_samples_per_second': 4.629, 'train_steps_per_second': 0.935, 'total_flos': 55584375634944.0, 'train_loss': 1.9712982536764705, 'epoch': 5.0})

### Evaluation

In [14]:
trainer.evaluate()

{'eval_loss': 1.8685897588729858,
 'eval_rouge1': 0.1113653434312454,
 'eval_rouge2': 0.030354299883526517,
 'eval_rougeL': 0.0871916761810557,
 'eval_rougeLsum': 0.10199213477198452,
 'eval_runtime': 23.6847,
 'eval_samples_per_second': 4.475,
 'eval_steps_per_second': 0.929,
 'epoch': 5.0}

In [19]:
# Hugging Face repository id
# huggingface hub model id
model_id="google/flan-t5-base-finetune"
dataset_id = "medical_data"
repository_id = f"{model_id.split('/')[1]}-{dataset_id}"

In [20]:
repository_id

'flan-t5-base-finetune-medical_data'

### Hugging Face Model Hub Integration

In [21]:
# Save our tokenizer and create model card
tokenizer.save_pretrained(repository_id)
trainer.create_model_card()

In [22]:
from huggingface_hub import notebook_login

notebook_login()

In [23]:
# Push the results to the hub
trainer.push_to_hub()

events.out.tfevents.1700141481.213b6bad9b79.927.0:   0%|          | 0.00/7.95k [00:00<?, ?B/s]

events.out.tfevents.1700141960.213b6bad9b79.927.1:   0%|          | 0.00/562 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

Upload 5 LFS files:   0%|          | 0/5 [00:00<?, ?it/s]

training_args.bin:   0%|          | 0.00/4.73k [00:00<?, ?B/s]

'https://huggingface.co/Omkar7/finetuned-t5-medical/tree/main/'

### Model Inference

In [24]:
from transformers import pipeline
from random import randrange

# Load the model and tokenizer from the Hugging Face hub using the pipeline
generator = pipeline(
    model="Omkar7/finetuned-t5-medical",
    tokenizer="Omkar7/finetuned-t5-medical",
    device=0  # Set to 0 or the appropriate GPU device index
)


(…)uned-t5-medical/resolve/main/config.json:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/990M [00:00<?, ?B/s]

(…)ical/resolve/main/generation_config.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

(…)dical/resolve/main/tokenizer_config.json:   0%|          | 0.00/20.8k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

(…)5-medical/resolve/main/added_tokens.json:   0%|          | 0.00/2.59k [00:00<?, ?B/s]

(…)cal/resolve/main/special_tokens_map.json:   0%|          | 0.00/2.54k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [25]:
# Sample input question
input_question = "What is (are) Glaucoma ?	"

In [26]:
# Generate an answer using the loaded model
generated_answer = generator(input_question, max_length=150, num_beams=5, length_penalty=0.6, early_stopping=True)

In [27]:
# Print the generated answer
print("Generated Answer:", generated_answer[0]['generated_text'])

Generated Answer: Glaucoma is the most common type of eye disease in the United States. It is the second leading cause of blindness in the United States.


### Model Comparison

In [34]:
# Sample input questions for evaluation
input_questions = ["What is (are) Glaucoma?", "What is High Blood Pressure ?","What causes Kidney Disease ","What is (are) Low Vision ?", "What is (are) Diabetes ?"]

# Load the base model
base_generator = pipeline(model="google/flan-t5-base", device=0)

# Generate answers using the base model
base_generated_answers = [base_generator(question, max_length=100, num_beams=5, length_penalty=0.8, early_stopping=True,temperature=0.5)[0]['generated_text']
                          for question in input_questions]

# Generate answers using the fine-tuned model
finetuned_generated_answers = [generator(question, max_length=100, num_beams=5, length_penalty=0.8, early_stopping=True,temperature=0.5)[0]['generated_text']
                               for question in input_questions]

# Print the results
print("Base Model Answers:")
for i, answer in enumerate(base_generated_answers):
    print(f"Question: {input_questions[i]}\nAnswer: {answer}\n")

print("\nFine-tuned Model Answers:")
for i, answer in enumerate(finetuned_generated_answers):
    print(f"Question: {input_questions[i]}\nAnswer: {answer}\n")


Base Model Answers:
Question: What is (are) Glaucoma?
Answer: ear infection

Question: What is High Blood Pressure ?
Answer: high blood pressure

Question: What causes Kidney Disease 
Answer: ureters

Question: What is (are) Low Vision ?
Answer: low vision

Question: What is (are) Diabetes ?
Answer: diabetes


Fine-tuned Model Answers:
Question: What is (are) Glaucoma?
Answer: Glaucoma is the most common type of eye disease in the United States. It is the second most common type of disease in the eye, behind macular degeneration and cataracts.

Question: What is High Blood Pressure ?
Answer: High blood pressure is a risk factor for heart disease, stroke, diabetes, and other health problems. The risk of developing high blood pressure rises with age. The risk of developing high blood pressure rises with age. The risk of developing high blood pressure increases with age.

Question: What causes Kidney Disease 
Answer: Kidney disease is a disease in which the kidneys do not produce enough b